In [1]:
import os
import sys
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import onnx
from onnxsim import simplify
import onnxruntime
from onnxruntime.quantization import (quantize_dynamic,
                                      QuantType,
                                      QuantFormat,
                                      quantize_static,
                                      CalibrationDataReader)

sys.path.append('../')
from platforms.core.config import cfg
from siamfcpp.pipeline.utils import (cxywh2xywh, get_crop,
                                     get_subwindow_tracking,
                                     xywh2cxywh, xyxy2cxywh)
from siamfcpp.model.common_opr.common_block import xcorr_depthwise

In [2]:
def to_bchw(im_patch):
    im_patch = im_patch.transpose(2, 0, 1)
    im_patch = im_patch[np.newaxis, :, :, :]
    return im_patch.astype(np.float32)

### Dynamic

In [2]:
backbone_init_fp32 = "../models/onnx/backbone_init.onnx"
backbone_init_uint8 = "../models/onnx_dynamic/backbone_init.onnx"

backbone_fp32 = "../models/onnx/backbone.onnx"
backbone_uint8 = "../models/onnx_dynamic/backbone.onnx"

head_fp32 = "../models/onnx/head.onnx"
head_uint8 = "../models/onnx_dynamic/head.onnx"

In [3]:
quantized_model = quantize_dynamic(backbone_init_fp32, backbone_init_uint8, weight_type=QuantType.QUInt8)
quantized_model = quantize_dynamic(backbone_fp32, backbone_uint8, weight_type=QuantType.QUInt8)
quantized_model = quantize_dynamic(head_fp32, head_uint8, weight_type=QuantType.QUInt8)

### Static

*backbone_init*

In [3]:
model_fp32 = "../models/onnx/backbone_init.onnx"
model_quant = "../models/onnx_static/backbone_init.onnx"

In [4]:
calibration_image_folder = "C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background"

In [5]:
def _preprocess_images(images_folder: str, size_limit=0):
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    unconcatenated_batch_data = []


    for image_name in tqdm_notebook(batch_filenames[:1]):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_z_crop, _ = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            avg_chans=avg_chans,
            context_amount=0.5,
            func_get_subwindow=get_subwindow_tracking,
        )

        im_z_crop = to_bchw(im_z_crop)

        unconcatenated_batch_data.append(im_z_crop)
        
    batch_data = np.concatenate(np.expand_dims(unconcatenated_batch_data, axis=0), axis=0)
    
    return batch_data


class DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str, model_path: str):
        self.enum_data = None

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])

        # Convert image to input data
        self.nchw_data_list = _preprocess_images(calibration_image_folder, size_limit=0)
        
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(self.nchw_data_list)
    

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(
                [{self.input_name: self.nchw_data_list[idx]} for idx in range(self.datasize)]
            )
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None

In [6]:
reader = DataReader(calibration_image_folder, model_fp32)

C:\Users\isd.illia.maliha\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
quantized_model = quantize_static(model_fp32,
                                  model_quant,
                                  reader,
                                  quant_format=QuantFormat.QDQ,
                                  activation_type=QuantType.QInt8,
                                  weight_type=QuantType.QInt8,)

-------

---------------------------

-----------------------------

*backbone*

In [8]:
model_fp32 = "../models/onnx/backbone.onnx"
model_quant = "../models/onnx_static/backbone.onnx"

In [9]:
calibration_image_folder = "C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background"

In [10]:
def _preprocess_images(images_folder: str, size_limit=0):
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    unconcatenated_batch_data = []


    for image_name in tqdm_notebook(batch_filenames[:1]):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_x_crop, scale_x = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            x_size=303,
            avg_chans=avg_chans,
            context_amount=cfg.context_amount,
            func_get_subwindow=get_subwindow_tracking,
        )

        im_x_crop = to_bchw(im_x_crop)

        unconcatenated_batch_data.append(im_x_crop)
        
    batch_data = np.concatenate(np.expand_dims(unconcatenated_batch_data, axis=0), axis=0)
    
    return batch_data


class DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str, model_path: str):
        self.enum_data = None

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])

        # Convert image to input data
        self.nchw_data_list = _preprocess_images(calibration_image_folder, size_limit=0)
        
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(self.nchw_data_list)
    

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(
                [{self.input_name: self.nchw_data_list[idx]} for idx in range(self.datasize)]
            )
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None

In [11]:
reader = DataReader(calibration_image_folder, model_fp32)

C:\Users\isd.illia.maliha\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
quantized_model = quantize_static(model_fp32,
                                  model_quant,
                                  reader,
                                  quant_format=QuantFormat.QDQ,
                                  activation_type=QuantType.QInt8,
                                  weight_type=QuantType.QInt8,)

-----------------------

--------------------

------------------

*head*

In [27]:
backbone_init_fp32 = "../models/onnx/backbone_init.onnx"
backbone_fp32 = "../models/onnx/backbone.onnx"
model_fp32 = "../models/onnx/head.onnx"
model_opt = "../models/onnx/head_opt.onnx"

model_quant = "../models/onnx_static/head.onnx"

In [28]:
calibration_image_folder = "C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background"

In [29]:
def _preprocess_images(backbone_init_path: str, backbone_path: str, images_folder: str, size_limit=0):
    
    bone_init = onnxruntime.InferenceSession(backbone_init_path, providers=['CPUExecutionProvider'])
    bone = onnxruntime.InferenceSession(backbone_path, providers=['CPUExecutionProvider'])
    
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    in1,in2 = [],[]
    

    for image_name in tqdm_notebook(batch_filenames[:1]):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_z_crop, _ = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            avg_chans=avg_chans,
            context_amount=0.5,
            func_get_subwindow=get_subwindow_tracking,
        )
        im_z_crop = to_bchw(im_z_crop)
        
        c_z_k, r_z_k = bone_init.run(None, {'input': im_z_crop})
    
    

        im_x_crop, scale_x = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            x_size=303,
            avg_chans=avg_chans,
            context_amount=cfg.context_amount,
            func_get_subwindow=get_subwindow_tracking,
        )
        im_x_crop = to_bchw(im_x_crop)
    
        c_x, r_x = bone.run(None, {'input': im_x_crop})
        
        c_out = xcorr_depthwise(torch.Tensor(c_x), torch.Tensor(c_z_k))
        r_out = xcorr_depthwise(torch.Tensor(r_x), torch.Tensor(r_z_k))
        
        in1.append(c_out.numpy())
        in2.append(r_out.numpy())

        
    batch_data1 = np.concatenate(np.expand_dims(in1, axis=0), axis=0)
    batch_data2 = np.concatenate(np.expand_dims(in2, axis=0), axis=0)
    
    return [batch_data1, batch_data2]


class DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str, model_path: str, backbone_init_path: str, backbone_path: str):
        self.enum_data = None

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])

        # Convert image to input data
        self.nchw_data_list = _preprocess_images(backbone_init_path, backbone_path, calibration_image_folder, size_limit=0)
        
        self.input_names = [session.get_inputs()[i].name for i in range(2)]
        self.datasize = len(self.nchw_data_list[0])
    

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(
                [{self.input_names[0]: self.nchw_data_list[0][idx],
                  self.input_names[1]: self.nchw_data_list[1][idx]} for idx in range(self.datasize)]
            )
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None

In [30]:
!python -m onnxruntime.quantization.preprocess --input "../models/onnx/head.onnx" --output "../models/onnx/head_opt.onnx"

In [31]:
reader = DataReader(calibration_image_folder, model_opt, backbone_init_fp32, backbone_fp32)

C:\Users\isd.illia.maliha\AppData\Local\Temp\ipykernel_9092\1704575551.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image_name in tqdm_notebook(batch_filenames[:1]):


  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
quantized_model = quantize_static(model_opt,
                                  model_quant,
                                  reader,
                                  quant_format=QuantFormat.QDQ,
                                  activation_type=QuantType.QInt8,
                                  weight_type=QuantType.QInt8,)

----------------------

#### One input

In [164]:
backbone_fp32 = "../networks/onnx/backbone_bottleneck_pe.onnx"
model_fp32 = "../networks/onnx/complete.onnx"
# model_one = "../networks/onnx/complete_one.onnx"
model_one = "../networks/onnx16/complete_one.onnx"

In [165]:
yaml_fname = '../experiments/stark_lightning_X_trt/baseline_rephead_4_lite_search5.yaml'
checkpoint_name = "../checkpoints/train/stark_lightning_X_trt/baseline_rephead_4_lite_search5/STARKLightningXtrt_ep0500.pth.tar"

update_config_from_file(yaml_fname)

model = build_stark_lightning_x_trt(cfg, phase='test')
model.load_state_dict(torch.load(checkpoint_name, map_location='cpu')['net'], strict=True)
model = repvgg_model_convert(model)
model.eval()

backbone = model.backbone
bottleneck = model.bottleneck
position_embed = model.pos_emb_x
transformer = model.transformer
box_head = model.box_head
box_head.coord_x = box_head.coord_x.cpu()
box_head.coord_y = box_head.coord_y.cpu()
torch_model = STARK(backbone, bottleneck, position_embed, transformer, box_head)

##########  Warning: Dilation is not valid in current code  ##########
Building lite transformer encoder...
head channel: 128


In [166]:
onnx_model = onnxruntime.InferenceSession(model_fp32, providers=['CPUExecutionProvider'])
torch_model.load_state_dict(torch.load('../networks/pytorch/complete.pth'))

<All keys matched successfully>

In [167]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def get_data(bs=1, sz_x=256, hw_z=64, c=256):
    img_x = torch.randn(bs, 3, sz_x, sz_x, requires_grad=True)
    mask_x = torch.rand(bs, sz_x, sz_x, requires_grad=True) > 0.5
    feat_vec_z = torch.randn(hw_z, bs, c, requires_grad=True)  # HWxBxC
    mask_vec_z = torch.rand(bs, hw_z, requires_grad=True) > 0.5  # BxHW
    pos_vec_z = torch.randn(hw_z, bs, c, requires_grad=True)  # HWxBxC
    return img_x, mask_x, feat_vec_z, mask_vec_z, pos_vec_z


bs = 1
sz_x = cfg.TEST.SEARCH_SIZE
hw_z = cfg.DATA.TEMPLATE.FEAT_SIZE ** 2
c = cfg.MODEL.HIDDEN_DIM

template, template_mask, feat_z, mask_z, pos_z = get_data(bs=bs, sz_x=sz_x, hw_z=hw_z, c=c)

In [168]:
xt = torch_model(template, template_mask, feat_z, mask_z, pos_z) 

xo = onnx_model.run(None, {'img_x': to_numpy(template),
                           'mask_x': to_numpy(template_mask), 
                           'feat_vec_z': to_numpy(feat_z), 
                           'mask_vec_z': to_numpy(mask_z),
                           'pos_vec_z': to_numpy(pos_z)})[0]

xt.detach().numpy().astype(float)-xo.astype(float)

array([[ 8.04662704e-07, -5.96046448e-08,  1.63912773e-06,
         1.19209290e-07]])

In [169]:
# class MyModel(nn.Module):
#     def __init__(self, main_model):
#         super(MyModel, self).__init__()
#         self.main_model = main_model
        
#     def forward(self, x):
#         x = x.reshape((-1,6,320,320))
#         img_x, x2 = torch.split(x, 3, dim=1)
        
#         mask_x, fp_z, mask_vec_z = torch.split(x2, 1, dim=1)
        
#         mask_x = mask_x[:,0,...]
        
#         mask_vec_z = mask_vec_z[:,0,159,128:-128]
#         mask_vec_z = mask_vec_z.type(torch.BoolTensor)
# #         mask_vec_z = mask_vec_z > 0.5
        
#         fp_z = fp_z[...,96:-96,96:-96]
#         feat_vec_z, pos_vec_z = torch.split(fp_z, 64, dim=2)
        
#         feat_vec_z = feat_vec_z[:,0,...]
#         feat_vec_z = feat_vec_z.permute((1,0,2))

#         pos_vec_z  = pos_vec_z[:,0,...]
#         pos_vec_z  = pos_vec_z.permute((1,0,2))

#         out = self.main_model(img_x, mask_x, feat_vec_z, mask_vec_z, pos_vec_z)

#         return out

class MyModel(nn.Module):
    def __init__(self, main_model):
        super(MyModel, self).__init__()
        self.main_model = main_model
        
    def forward(self, x1,x2,x3,x4,x5):
        
        x2 = x2.type(torch.BoolTensor)
        x4 = x4.type(torch.BoolTensor)
        out = self.main_model(x1,x2,x3,x4,x5)

        return out

In [170]:
img_x = template.clone()
mask_x = template_mask.clone()
feat_vec_z = feat_z.clone()
mask_vec_z = mask_z.clone()
pos_vec_z = pos_z.clone()

# mask_x = mask_x[:,None]

# feat_vec_z = feat_vec_z.permute((1,0,2))[:,None]
# pos_vec_z  = pos_vec_z.permute((1,0,2))[:,None]
# fp_z = torch.cat([feat_vec_z, pos_vec_z], dim=2)
# fp_z = F.pad(fp_z,(96,96,96,96))

# mask_vec_z = mask_vec_z[:, None, None]
# mask_vec_z = F.pad(mask_vec_z,(128,128,159,160))

# inp = torch.cat([img_x, mask_x, fp_z, mask_vec_z], dim=1)
# inp = inp.reshape((1,3,320,640))

In [171]:
model = MyModel(torch_model)
# x_new = model(inp)
x_new = model(img_x, mask_x.type(torch.FloatTensor), feat_vec_z, mask_vec_z.type(torch.FloatTensor), pos_vec_z)

torch.sum(xt-x_new), torch.sum(torch.Tensor(xo)-x_new)

(tensor(0., grad_fn=<SumBackward0>),
 tensor(-2.5034e-06, grad_fn=<SumBackward0>))

In [172]:
xt,torch.Tensor(xo),x_new

(tensor([[0.4432, 0.4292, 0.2356, 0.1382]], grad_fn=<StackBackward>),
 tensor([[0.4432, 0.4292, 0.2356, 0.1382]]),
 tensor([[0.4432, 0.4292, 0.2356, 0.1382]], grad_fn=<StackBackward>))

In [173]:
torch.onnx.export(model, (img_x, mask_x.type(torch.FloatTensor), feat_vec_z, mask_vec_z.type(torch.FloatTensor), pos_vec_z),
                  model_one, 
                  input_names=['img_x', 'mask_x', 'feat_vec_z', 'mask_vec_z', 'pos_vec_z'],
                  output_names=['outputs_coord'],
                  verbose=True, opset_version=11, export_params=True, 
                  do_constant_folding=True) 

# simplified_model, check = simplify(model_one, skip_fuse_bn=False)
# onnx.save_model(simplified_model, model_one)

graph(%img_x : Float(1:307200, 3:102400, 320:320, 320:1, requires_grad=1, device=cpu),
      %mask_x : Float(1:102400, 320:320, 320:1, requires_grad=0, device=cpu),
      %feat_vec_z : Float(64:128, 1:128, 128:1, requires_grad=1, device=cpu),
      %mask_vec_z : Float(1:64, 64:1, requires_grad=0, device=cpu),
      %pos_vec_z : Float(64:128, 1:128, 128:1, requires_grad=1, device=cpu),
      %main_model.backbone.body.stage0.rbr_reparam.weight : Float(48:27, 3:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %main_model.backbone.body.stage0.rbr_reparam.bias : Float(48:1, requires_grad=0, device=cpu),
      %main_model.backbone.body.stage1.0.rbr_reparam.weight : Float(48:432, 48:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %main_model.backbone.body.stage1.0.rbr_reparam.bias : Float(48:1, requires_grad=0, device=cpu),
      %main_model.backbone.body.stage1.1.rbr_reparam.weight : Float(48:432, 48:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %main_model.backbone.body.stage1.1.rbr_rep

In [163]:
# onnx_model = onnxruntime.InferenceSession(model_one, providers=['CPUExecutionProvider'])

###### Quant

In [14]:
model_quant = "../networks/onnx/complete_one_int8_static.onnx"

In [15]:
calibration_image_folder = "C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background"

In [16]:
def _preprocess_images(backbone_path: str, images_folder: str, size_limit=0):
    bone = onnxruntime.InferenceSession(backbone_path, providers=['CPUExecutionProvider'])
    inp_names = [bone.get_inputs()[i].name for i in range(2)]
    
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
    in1,in2,in3,in4,in5 = [],[],[],[],[]
 
    preprocessor = PreprocessorX_onnx()

    for image_name in tqdm_notebook(batch_filenames[:]):
        image_filepath = images_folder + "/" + image_name
        image = cv2.cvtColor(cv2.imread(image_filepath), cv2.COLOR_BGR2RGB).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(int(0.4*w), int(0.6*w))
        y = np.random.randint(int(0.4*h), int(0.6*h))
        ww = int(0.1*w)
        hh = int(0.1*h)
        
        z_patch_arr, _, z_amask_arr = sample_target(image.copy(), [x,y,ww,hh], 2.0, 128)
        template, template_mask = preprocessor.process(z_patch_arr, z_amask_arr)
        
        inp_dict = {inp_names[0]: template,
                    inp_names[1]: template_mask}
        bone_out = bone.run(None, inp_dict)
    
        x_patch_arr, _, x_amask_arr = sample_target(image.copy(), [x,y,ww,hh], 5.0, output_sz=320)
        template, template_mask = preprocessor.process(x_patch_arr, x_amask_arr)
        
        img_x = torch.Tensor(template)
        mask_x = torch.Tensor(template_mask) 
        feat_vec_z = torch.Tensor(bone_out[0])
        mask_vec_z = torch.Tensor(bone_out[1])
        pos_vec_z = torch.Tensor(bone_out[2])

        mask_x = mask_x[:,None]

        feat_vec_z = feat_vec_z.permute((1,0,2))[:,None]
        pos_vec_z  = pos_vec_z.permute((1,0,2))[:,None]
        fp_z = torch.cat([feat_vec_z, pos_vec_z], dim=2)
        fp_z = F.pad(fp_z,(96,96,96,96))

        mask_vec_z = mask_vec_z[:, None, None]
        mask_vec_z = F.pad(mask_vec_z,(128,128,159,160))

        inp = torch.cat([img_x, mask_x, fp_z, mask_vec_z], dim=1)
        inp = inp.reshape((1,3,320,640))

        in1.append(inp.numpy())


    batch_data1 = np.concatenate(np.expand_dims(in1, axis=0), axis=0)
    
#     print(batch_data1.shape)
    
    return batch_data1


class DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str, model_path: str, backbone_path: str):
        self.enum_data = None

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])

        # Convert image to input data
        self.nchw_data_list = _preprocess_images(backbone_path, calibration_image_folder, size_limit=0)
        
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(self.nchw_data_list)
    

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(
                [{self.input_name: self.nchw_data_list[idx]} for idx in range(self.datasize)]
            )
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None

In [17]:
# !python -m onnxruntime.quantization.preprocess --input "../networks/onnx/complete_one.onnx" --output "../networks/onnx/complete_one.onnx"

In [18]:
reader = DataReader(calibration_image_folder, model_one, backbone_fp32)

In [19]:
quantized_model = quantize_static(model_one,
                                  model_quant,
                                  reader,
                                  quant_format=QuantFormat.QDQ,
                                  activation_type=QuantType.QInt8,
                                  weight_type=QuantType.QInt8,)

In [20]:
simplified_model, check = simplify(model_quant, skip_fuse_bn=False)
onnx.save_model(simplified_model, model_quant)

In [21]:
session = onnxruntime.InferenceSession(model_quant, providers=['CPUExecutionProvider'])

for x in session.get_outputs():
    print(x.name)